# Finetuning API 호출
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. API Key 설정
2. 파일 변환
3. 파일 업로드
4. Finetuning 작업 요청
5. 작업 진행상황 확인 
6. 모델명 확인
7. 모델 사용 및 비교

### 관련 링크
- [OpenAI FineTuning Docs](https://platform.openai.com/docs/guides/fine-tuning)
- [OpenAI FineTuning API](https://platform.openai.com/docs/api-reference/fine-tuning)

In [ ]:
!pip install openai 

In [ ]:
# 1. 환경 설정

import os
from openai import OpenAI

# 환경 변수 설정 - Finetuning
os.environ["OPENAI_API_KEY"] = "sk-proj-mnjdCgEPkq1DCQ6oGvHyT3BlbkFJikgeB1O23L1ttMnAHRYu"

client = OpenAI()

##### Finetuning Completion Model 

In [ ]:
# 2. 파일 변환

!openai tools fine_tunes.prepare_data --file FT_InitialsGame_Train.csv --quiet

In [ ]:
# 3. 파일 업로드

training_file_name = "FT_InitialsGame_Train_prepared.jsonl"

training_response = client.files.create(
    file=open(training_file_name, "rb"), 
    purpose="fine-tune"
)

training_file_id = training_response.id

print(training_file_id)

In [ ]:
# 4. Finetning 작업 요청

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="",
    suffix="이니셜-번호",
    hyperparameters={"n_epochs":4}
)

job_id = response.id

print(job_id)

In [ ]:
# 5. 작업 진행상황 확인

model_response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_status = model_response.status
print(fine_tuned_model_status)

In [ ]:
# 6. 작업 완료된 모델명 확인

response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

print("Fine-tuned model ID:", fine_tuned_model_id)

In [ ]:
# 7-1. 베이스 모델 사용

completion = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    temperature=0.5,
    max_tokens=10,
    stop=["\n"],
    prompt="ㅈㄱ ->"
)

completion.choices[0].text

In [ ]:
# 7-2. Finetuned 모델 사용

completion = client.completions.create(
    model="ft:davinci-002:mijeongjeon:2024-jmj:8vkMFHij",
    temperature=0.5,
    max_tokens=10,
    stop=["\n"],
    prompt="ㅎㅁ ->"
)

completion.choices[0].text

##### Finetuning ChatCompletion Model 

In [ ]:
# 8. 파일 업로드

chat_training_file_name = "FT_Food_Info.jsonl"

chat_training_response = client.files.create(
    file=open(chat_training_file_name, "rb"),
    purpose="fine-tune"
)

chat_training_file_id = chat_training_response.id
print(chat_training_file_id)

In [ ]:
# 9. 작업 요청
chat_response = client.fine_tuning.jobs.create(
    training_file=chat_training_file_id,
    model="",
    suffix="이니셜-번호",
    hyperparameters={"n_epochs":4}
)

chat_job_id = chat_response.id

print(chat_job_id)

In [ ]:
# 10. 작업 진행상황 확인

chat_model_response = client.fine_tuning.jobs.retrieve(chat_job_id)
chat_fine_tuned_model_status = chat_model_response.status
print(chat_fine_tuned_model_status)

In [ ]:
# 11. 작업 완료된 모델명 확인

chat_response = client.fine_tuning.jobs.retrieve(chat_job_id)
chat_fine_tuned_model_id = chat_response.fine_tuned_model

print("Fine-tuned model ID:", chat_fine_tuned_model_id)

In [ ]:
# 12-1. Base 모델 사용

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system", "content":"먹을 수 있는 음식 이름을 알려주면 칼로리양과 주요 영양소 탄수화물, 단백질, 지방 함량을 알려주고 음식 이모지를 붙여주는 친절한 도우미"},
        {"role":"user", "content":"떡볶이"}
    ]
)

print(response.choices[0].message.content)


In [ ]:
# 12-2. Finetuned 모델 사용

response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0613:mijeongjeon:2024-jmj:8w3Mz4RO",
    messages=[
        {"role":"system", "content":"먹을 수 있는 음식 이름을 알려주면 칼로리양과 주요 영양소 탄수화물, 단백질, 지방 함량을 알려주고 음식 이모지를 붙여주는 친절한 도우미"},
        {"role":"user", "content":"떡볶이"}
    ]
)

print(response.choices[0].message.content)